The data downloaded using the script "July_to_Aug23_data_download.py" gives non-uniformly resampled data

For this we first need to linearly interpolate to 8s 

Below given is the procedure on how to accomplish the same

In [1]:
import os
import pandas as pd

In [2]:
# specify the input path
ip_path = "/Users/harleenkaur/Documents/phyFai_thresholds_decision/UPS_download_interpolation/July_to_Aug23_csvs/"
# specify the path where you want to download
out_path = "/Users/harleenkaur/Documents/phyFai_thresholds_decision/UPS_download_interpolation/July_to_Aug23_interpolated_data/"
files = ['b173E8.csv']

In [53]:
def read_and_filter_data(file_path, diagnostic_key):
    df = pd.read_csv(file_path)
    filtered_df = df[df['diagnostic'] == diagnostic_key]
    return filtered_df


def process_and_interpolate(df, new_col_name, scale_factor=1, freq='8s'):
    
    
    df['dateTime'] = pd.to_datetime(df['dateTime'], utc=True)
    df = df.sort_values('dateTime')
    df = df[['dateTime', 'data']]
    df.columns = ['dateTime', new_col_name]

    if scale_factor != 1:
        df[new_col_name] = df[new_col_name] / scale_factor

    df.set_index('dateTime', inplace=True)
    df = df[~df.index.duplicated(keep='first')]
    full_index = df.index.union(pd.date_range(start=df.index[0], end=df.index[-1], freq=freq))
    df_interpolated = df.reindex(full_index.sort_values()).interpolate('linear')
    df_interpolated = df_interpolated.loc[pd.date_range(start=df.index[0], end=df.index[-1], freq=freq)]
    df_interpolated = df_interpolated.reset_index().rename(columns={'index': 'datetime_utc'})

    return df_interpolated

In [50]:
for file_name in files:
    file_path = os.path.join(ip_path, file_name)

    df_ect = read_and_filter_data(file_path, 'DiagnosticEngineCoolantTemperatureId')
    df_oilp = read_and_filter_data(file_path, 'DiagnosticOilPressureId')
    df_rpm = read_and_filter_data(file_path, 'DiagnosticEngineSpeedId')

    if not df_oilp.empty and not df_rpm.empty and not df_ect.empty:
        df_oilp_interpolated = process_and_interpolate(df_oilp, 'OilP', scale_factor=1000)
        df_rpm_interpolated = process_and_interpolate(df_rpm, 'ENGINE RPM')
        df_ect_interpolated = process_and_interpolate(df_ect, 'ECT')

        # Now, df_ect_interpolated, df_oilp_interpolated, df_rpm_interpolated
        # are ready for further processing or merging


DatetimeIndex(['2025-08-05 13:57:26.927000+00:00',
               '2025-08-05 13:57:29.927000+00:00',
               '2025-08-05 13:57:34.927000+00:00',
               '2025-08-05 13:57:42.927000+00:00',
               '2025-08-05 13:57:50.927000+00:00',
               '2025-08-05 13:57:58.927000+00:00',
               '2025-08-05 13:58:01.023000+00:00',
               '2025-08-05 13:58:06.530000+00:00',
               '2025-08-05 13:58:06.927000+00:00',
               '2025-08-05 13:58:13.040000+00:00',
               ...
               '2025-08-23 15:35:50.927000+00:00',
               '2025-08-23 15:35:53.120000+00:00',
               '2025-08-23 15:35:58.927000+00:00',
               '2025-08-23 15:36:03.137000+00:00',
               '2025-08-23 15:36:06.927000+00:00',
               '2025-08-23 15:36:14.927000+00:00',
               '2025-08-23 15:36:22.927000+00:00',
               '2025-08-23 15:36:30.927000+00:00',
               '2025-08-23 15:36:38.927000+00:00',
            